### Federal Reserve Economic Data (FRED)


In [4]:
import pandas as pd
import requests


def fetch_fred_data(series_id, api_key):
    url = f"https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={api_key}&file_type=json"
    response = requests.get(url).json()
    data = pd.DataFrame(response['observations'])
    data['date'] = pd.to_datetime(data['date'])
    # Remove any non-numeric values and convert to float
    data['value'] = pd.to_numeric(data['value'], errors='coerce')
    return data

api_key = '498e896779ad57e66a70de7fd117115c'

series_ids = {
    'IPI': 'INDPRO',
    'CPI': 'CPIAUCSL',
    'M1': 'M1SL',
    'M2': 'M2SL',
    'Housing Price Index': 'CSUSHPINSA',
    '15 Year Mortgage Rate': 'MORTGAGE15US',
    '30 Year Mortgage Rate': 'MORTGAGE30US',
    'Federal Funds Rate': 'FEDFUNDS'
}

dataframes = {}
for name, series_id in series_ids.items():
    dataframes[name] = fetch_fred_data(series_id, api_key)

# Example to view CPI data
print(dataframes['CPI'].head())

  realtime_start realtime_end       date  value
0     2024-06-12   2024-06-12 1947-01-01  21.48
1     2024-06-12   2024-06-12 1947-02-01  21.62
2     2024-06-12   2024-06-12 1947-03-01  22.00
3     2024-06-12   2024-06-12 1947-04-01  22.00
4     2024-06-12   2024-06-12 1947-05-01  21.95


In [42]:
df_ipi =dataframes['IPI'].drop(columns= ['realtime_start','realtime_end'])
df_CPI =dataframes['CPI'].drop(columns= ['realtime_start','realtime_end'])
df_m1 =dataframes['M1'].drop(columns= ['realtime_start','realtime_end'])
df_m2 =dataframes['M2'].drop(columns= ['realtime_start','realtime_end'])
df_hpi =dataframes['Housing Price Index'].drop(columns= ['realtime_start','realtime_end'])
df_15m =dataframes['15 Year Mortgage Rate'].drop(columns= ['realtime_start','realtime_end'])
df_30m =dataframes['30 Year Mortgage Rate'].drop(columns= ['realtime_start','realtime_end'])
df_ffr = df_ffr.drop(columns= ['realtime_start','realtime_end'])

In [43]:
df_ipi.rename(columns = {'value':'IPI'}, inplace = True)
df_CPI.rename(columns = {'value':'CPI'}, inplace = True)
df_m1.rename(columns = {'value':'M1'}, inplace = True)
df_m2.rename(columns = {'value':'M2'}, inplace = True)
df_hpi.rename(columns = {'value':'Housing Price Index'}, inplace = True)
df_15m.rename(columns = {'value':'15 Year Mortgage Rate'}, inplace = True)
df_30m.rename(columns = {'value':'30 Year Mortgage Rate'}, inplace = True)
df_ffr.rename(columns = {'value':'Federal Funds Rate'}, inplace = True)

In [44]:
df_ipi.set_index('date', inplace=True)
df_CPI.set_index('date', inplace=True)
df_m1.set_index('date', inplace=True)
df_m2.set_index('date', inplace=True)
df_hpi.set_index('date', inplace=True)
df_15m.set_index('date', inplace=True)
df_30m.set_index('date', inplace=True)
df_ffr.set_index('date', inplace=True)
df_ei_m = df_ipi.join([df_CPI,df_m1,df_m2,df_hpi, df_ffr],how = 'outer')
df_ei_w = df_15m.join(df_30m, how = 'outer')

In [28]:
df_30m

,date,value
0,1971-04-02,7.33
1,1971-04-09,7.31
2,1971-04-16,7.31
3,1971-04-23,7.31
4,1971-04-30,7.29
...,...,...
2772,2024-05-16,7.02
2773,2024-05-23,6.94
2774,2024-05-30,7.03
2775,2024-06-06,6.99


### Gold Price
https://data.nasdaq.com/tools/api

In [5]:
# %pip install quandl
import quandl

quandl.ApiConfig.api_key = '4wWVxPhofJuUo2QY-Tm_'

gold_data = quandl.get("LBMA/GOLD")

# # Check existing columns before renaming
# print("Columns before renaming:", gold_data.columns)

# Ensure correct column renaming
if 'USD (AM)' in gold_data.columns:
    gold_data.reset_index(inplace=True)
    gold_data.rename(columns={'USD (AM)': 'Gold Price'}, inplace=True)
    gold_data['Gold Price'] = pd.to_numeric(gold_data['Gold Price'], errors='coerce')

In [9]:
gold_data = gold_data.drop(['USD (PM)','GBP (AM)','GBP (PM)','EURO (AM)','EURO (PM)'], axis =1)

In [36]:
gold_data.rename(columns = {'Date':'date'}, inplace = True)
gold_data.set_index('date', inplace=True)

In [37]:
df_ei_d = gold_data

In [45]:
df_ei_d.to_csv('economic_indicators_daily.csv')
df_ei_w.to_csv('economic_indicators_weekly.csv')
df_ei_m.to_csv('economic_indicators_monthly.csv')